
# Data Analytics Project: Data Cleaning and Feature Engineering

This notebook reorganizes and documents the steps taken to prepare and analyze the nutrition and wellbeing survey data.

The workflow includes:

1. **Loading and inspecting the data**
2. **Cleaning and formatting columns**
3. **Handling missing values**
4. **Computing composite scores** (Food Insecurity, Resilience, Stress Mindset, Psychological Distress, Motivation, Eating habits, etc.)
5. **Deriving additional features** (e.g., healthy lifestyle, diet adherence)
6. **Saving the cleaned dataset**

Each code cell contains detailed comments explaining the purpose and logic behind the operation to make the notebook easy to follow.


In [ ]:

# Import required libraries
import pandas as pd
import numpy as np

# Set pandas option to avoid silent downcasting warnings
pd.set_option('future.no_silent_downcasting', True)

# Replace this path with the location of your Excel file
data_path = '/content/Nutrition with no None.xlsx'

# Load the survey data into a DataFrame
# If you're working in a different environment, adjust the path accordingly.
df = pd.read_excel(data_path)

# Display the first few rows to verify successful loading
df.head()


In [ ]:

# Inspect the basic structure of the dataset
print('Shape of the raw data:', df.shape)
print('
Column information:')
df.info()



## Data Cleaning: Standardizing column names

The survey data column names may contain leading or trailing whitespace.
We strip whitespace to ensure consistent referencing throughout the notebook.


In [ ]:

# Remove leading/trailing whitespace from column names
# This simplifies column selection later in the analysis
df.columns = df.columns.str.strip()

# Verify that whitespace has been removed
print('Updated column names:')
print(df.columns.tolist())



## Data Cleaning: Harmonizing age categories

The `"What is your age?"` column contains overlapping and inconsistent age ranges (e.g. "18-22" and "22-29").
We map similar ranges into unified categories for consistency.


In [ ]:

# Define mapping to harmonize age ranges
age_mapping = {
    '18-22': '18-22',
    '22-29': '23-29',  # combine 22-29 into 23-29 to avoid overlap
    '23-29': '23-29',
    '30-39': '30-39',
    '40-49': '40-49',
    '50-59': '50-59',
    '60 or older': '60+',
    'Under 18': 'Under 18'
}

# Apply the mapping to create a cleaned age column
df['What is your age?'] = df['What is your age?'].replace(age_mapping)

# Inspect the unique age categories after harmonization
print('Unique age categories after cleaning:')
print(sorted(df['What is your age?'].unique()))



## Data Cleaning: Region and Dietary Style

We strip extra spaces and convert text to lowercase in two columns:

- **`What region do you live in ?`**: trailing whitespace removed.
- **`What is your typical dietary style?`**: trimmed and lower‑cased to normalize responses.


In [ ]:

# Clean region column by stripping whitespace
df['What region do you live in ?'] = df['What region do you live in ?'].str.strip()

# Clean dietary style column by stripping whitespace and converting to lowercase
df['What is your typical dietary style?'] = (
    df['What is your typical dietary style?']
    .str.strip()
    .str.lower()
)

# Inspect unique values after cleaning
print('Regions:', df['What region do you live in ?'].unique())
print('
Dietary styles:', df['What is your typical dietary style?'].unique())



## Handling Missing Values

To ensure the quality of the analysis, we perform the following steps:

1. **Row filtering** – Remove respondents with more than 30% missing responses.
2. **FIES columns** – Replace missing values in the Food Insecurity Experience Scale (FIES) items with "Not sure or don't know".
3. **Other columns** – Impute missing values using the mode of each column.


In [ ]:

# Set a threshold for the maximum allowed proportion of missing values per row
missing_threshold = 0.30

# Compute the proportion of missing values per row
row_missing_ratio = df.isna().sum(axis=1) / df.shape[1]

# Keep only rows with missing proportion less than or equal to the threshold
df_clean = df[row_missing_ratio <= missing_threshold].copy()

print('Number of rows before filtering:', df.shape[0])
print('Number of rows after filtering:', df_clean.shape[0])

# Define FIES columns (Food Insecurity Experience Scale)
fies_cols = [
    'During the last 12 months, was there a time when, because of lack of money or other resources you were worried you would not have enough food to eat?',
    'During the last 12 months, was there a time when, because of lack of money or other resources you were unable to eat healthy and nutritious food?',
    'During the last 12 months, was there a time when, because of lack of money or other resources you ate only a few kinds of foods?',
    'During the last 12 months, was there a time when, because of lack of money or other resources you had to skip a meal?',
    'During the last 12 months, was there a time when, because of lack of money or other resources you ate less than you thought you should?',
    'During the last 12 months, was there a time when, because of lack of money or other resources your household ran out of food?',
    'During the last 12 months, was there a time when, because of lack of money or other resources you were hungry but did not eat?',
    'During the last 12 months, was there a time when, because of lack of money or other resources you went without eating for a whole day?'
]

# Fill missing FIES responses with a neutral option
fies_fill_value = "Not sure or don't know"
df_clean[fies_cols] = df_clean[fies_cols].fillna(fies_fill_value)

# For all other columns, impute missing values with the modal category
cols_to_fill = [col for col in df_clean.columns if col not in fies_cols]
for col in cols_to_fill:
    if df_clean[col].isnull().any():
        mode_value = df_clean[col].mode()[0]
        df_clean[col] = df_clean[col].fillna(mode_value)

# Verify that no missing values remain
print('
Remaining missing values per column:')
print(df_clean.isna().sum().sort_values(ascending=False).head())



## Computing the Food Insecurity (FIES) Score

The FIES consists of eight binary questions.
We map responses to numerical values (Yes=1, No=0, Not sure=0.5) and sum across items to obtain the FIES score for each respondent.


In [ ]:

# Map FIES responses to numeric values
fies_mapping = {
    'Yes': 1,
    'No': 0,
    "Not sure or don't know": 0.5
}

# Convert FIES responses into numeric format
df_clean[fies_cols] = df_clean[fies_cols].replace(fies_mapping)

# Compute the FIES score by summing across the eight items
df_clean['FIES_Score'] = df_clean[fies_cols].sum(axis=1)

# Display summary statistics of the FIES score
print(df_clean['FIES_Score'].describe())



## Computing the Resilience (RS) Score

Resilience is assessed using six statements with responses on a 1–5 scale.
Negative items are reverse‑scored (i.e., 6 – response value) so that higher scores indicate greater resilience.
The RS score is the sum of the six items.


In [ ]:

# List of all resilience items (as they appear in the survey)
rs_items = [
    '1. I tend to bounce back quickly after hard times',
    '2. I have a hard time making it through stressful events',
    '3. It does not take me long to recover from a stressful event.',
    '4. It is hard for me to snap back when something bad happens.',
    '5. I usually come through difficult times with little trouble.',
    '6. I tend to take a long time to get over set-backs in my life.'
]

# Identify negative items that should be reverse‑scored
negative_rs = [rs_items[1], rs_items[3], rs_items[5]]

# Reverse‑score the negative items (scale is 1–5, so 6 - x)
df_clean[negative_rs] = 6 - df_clean[negative_rs]

# Sum all RS items to compute the Resilience score
df_clean['RS_Score'] = df_clean[rs_items].sum(axis=1)

print(df_clean['RS_Score'].describe())



## Computing the Stress Mindset (SMM) Score

The Stress Mindset Measure consists of four positive and four negative statements.
Responses range from 'strongly disagree' (0) to 'strongly agree' (4).
Negative items are reverse‑scored, and the total SMM score is the sum of all eight items.


In [ ]:

# Define positive and negative stress mindset items
sm_pos = [
    'Experiencing stress facilitates my learning and growth.',
    'Experiencing stress enhances my performance and productivity.',
    'Experiencing stress improves my health and vitality.',
    'The effects of stress are positive and should be utilized.'
]

sm_neg = [
    'The effects of stress are negative and should be avoided.',
    'Experiencing stress depletes my health and vitality.',
    'Experiencing stress inhibits my learning and growth.',
    'Experiencing stress debilitates my performance and productivity.'
]

# Mapping of text responses to numeric scale
smm_mapping = {
    'strongly disagree': 0,
    'disagree': 1,
    'neither agree nor disagree': 2,
    'neither agree nor disagree ': 2,  # handle stray spaces
    'neither agree nor disagree  ': 2,
    'agree': 3,
    'strongly agree': 4
}

# Clean and map stress mindset responses to numeric values
df_clean[sm_pos + sm_neg] = (
    df_clean[sm_pos + sm_neg]
    .apply(lambda col: col.str.strip().str.lower().replace(smm_mapping))
)

# Reverse‑score negative items (4 - x)
df_clean[sm_neg] = 4 - df_clean[sm_neg]

# Compute the SMM score by summing across the eight items
df_clean['SMM_Score'] = df_clean[sm_pos + sm_neg].sum(axis=1)

print(df_clean['SMM_Score'].describe())



## Computing the Kessler Psychological Distress (K6) Score

The K6 scale measures nonspecific psychological distress over the past 30 days.
Responses range from 'none of the time' to 'all of the time'.
We recode each response to a numeric value (0–4) and sum across the six items to produce the K6 score.


In [ ]:

# List of K6 items (questions about feelings over the past 30 days)
k6_items = [
    "About how often during the past 30 days did you feel nervous - would you say all of the time, most of the time, some of the time, a little of the time, or none of the time?",
    "During the past 30 days, about how often did you feel hopeless - all of the time, most of the time, some of the time, a little of the time, or none of the time?",
    "During the past 30 days, about how often did you feel restless or fidgety?",
    "How often did you feel so depressed that nothing could cheer you up?",
    "During the past 30 days, about how often did you feel that everything was an effort?",
    "During the past 30 days, about how often did you feel worthless?"
]

# Clean and lower-case the responses
for item in k6_items:
    df_clean[item] = df_clean[item].str.strip().str.lower()

# Mapping of K6 responses to numeric values
k6_mapping = {
    'none of the time': 0,
    'a little': 1,
    'a little of the time': 1,
    'some': 2,
    'some of the time': 2,
    'most': 3,
    'most of the time': 3,
    'all': 4,
    'all of the time': 4
}

# Apply the mapping to recode responses
df_clean[k6_items] = df_clean[k6_items].replace(k6_mapping)

# Compute the K6 score
df_clean['K6_Score'] = df_clean[k6_items].sum(axis=1)

print(df_clean['K6_Score'].describe())



## Defining Motivation and Eating Frequency Items

To compute composite scores for different motivational constructs and eating habits, we first define lists of survey questions corresponding to each construct.
These lists will be reused later to calculate scores.


In [ ]:

# Intrinsic Motivation Items: students learn because they enjoy the process
intrinsic_motivation = [
    '2. Because I experience pleasure and satisfaction while learning new things.',
    '4. For the intense feelings I experience when I am communicating my own ideas to others.',
    '6. For the pleasure I experience while surpassing myself in my studies.',
    '9. For the pleasure I experience when I discover new things never seen before.',
    '11. For the pleasure that I experience when I read interesting authors.',
    '13. For the pleasure that I experience while I am surpassing myself in one of my personal accomplishments.',
    '16. For the pleasure that I experience in broadening my knowledge about subjects which appeal to me.',
    '18. For the pleasure that I experience when I feel completely absorbed by what certain authors have written.',
    '20. For the satisfaction I feel when I am in the process of accomplishing difficult academic activities.',
    '23. Because my studies allow me to continue to learn about many things that interest me.',
    '25. For the "high" feeling that I experience while reading about various interesting subjects.',
    '27. Because college allows me to experience a personal satisfaction in my quest for excellence in my studies.'
]

# Extrinsic Motivation Items: students learn for external rewards or outcomes
extrinsic_motivation = [
    '1. Because with only a high-school degree I would not find a high-paying job later on.',
    '3. Because I think that a college education will help me better prepare for the career I have chosen.',
    '7. To prove to myself that I am capable of completing my college degree.',
    '8. In order to obtain a more prestigious job later on.',
    '10. Because eventually it will enable me to enter the job market in a field that I like.',
    '14. Because of the fact that when I succeed in college I feel important.',
    '15. Because I want to have "the good life" later on.',
    '17. Because this will help me make a better choice regarding my career orientation.',
    '21. To show myself that I am an intelligent person.',
    '22. In order to have a better salary later on.',
    '24. Because I believe that a few additional years of education will improve my competence as a worker.',
    '28. Because I want to show myself that I can succeed in my studies.'
]

# Amotivation Items: students lack purpose or interest in studying
amotivation_items = [
    "5. Honestly, I don't know; I really feel that I am wasting my time in school.",
    '12. I once had good reasons for going to college; however, now I wonder whether I should continue.',
    "19. I can't see why I go to college and frankly, I couldn't care less.",
    "26. I don't know; I can't understand what I am doing in school."
]

# Healthy eating frequency items: higher scores = healthier diet
healthy_eating = [
    'On an average week, how many times do you consume whole grain products? (e.g., whole grain bread, whole grain cereal, brown rice or whole wheat pasta',
    'On an average week, how many times do you consume dairy products?(e.g., milk, cheese, yogurt …etc.)',
    'On an average week, how many times do you consume fruits?',
    'On an average week, how many times do you consume nuts, including flaxseed?',
    'On an average week, how many times do you consume dark green leafy vegetables?(e.g., spinach, kale, lettuce …etc.)',
    'On an average week, how many times do you consume beans?(e.g. black beans, black-eyed peas, lentils, lima beans, soy nuts …etc.)',
    'On an average week, how many times do you consume fish and/or shellfish (including sardines and tuna)?'
]

# Neutral eating frequency items: general dietary indicators
neutral_items = [
    'In the past 7 days, how many times did you eat breakfast?',
    'On an average week, how many times do you consume rice and/or pasta?',
    'On average week, how many times do you take multivitamin supplements?',
    'On average week, how many times do you take fish oil supplements?',
    'On an average week, how many times do you consume red meat, chicken, or turkey?'
]

# Unhealthy eating frequency items: fast foods and caffeinated drinks
unhealthy_eating = [
    'On an average week, how many times do you consume coffee or other sources of caffeine?(e.g., soda, tea, chocolate, energy drink …etc.)',
    'On an average week, how many times do you consume fast foods and/or pre-made or packaged food?'
]

# Perceived Stress Scale (PSS) items: measure perceived stress during the past month
perceived_stress_items = [
    'In the last month, how often have you been upset because of something that happened unexpectedly?',
    'In the last month, how often have you felt that you were unable to control the important things in your life?',
    'In the last month, how often have you felt nervous and stressed?',
    'In the last month, how often have you felt confident about your ability to handle your personal problems?',
    'In the last month, how often have you felt that things were going your way?',
    'In the last month, how often have you found that you could not cope with all the things that you had to do?',
    'In the last month, how often have you been able to control irritations in your life?',
    'In the last month, how often have you felt that you were on top of things?',
    'In the last month, how often have you been angered because of things that happened that were outside of your control?',
    'In the last month, how often have you felt difficulties were piling up so high that you could not overcome them?'
]



## Computing Motivation Scores

We calculate aggregate scores for intrinsic motivation (IM), extrinsic motivation (EM), and amotivation (AM) by summing the responses to their respective items.
This step must be performed after ensuring that all items are numeric (e.g. via prior encoding).


In [ ]:

# Sum responses to compute motivation scores
df_clean['IM Score'] = df_clean[intrinsic_motivation].sum(axis=1)
df_clean['EM Score'] = df_clean[extrinsic_motivation].sum(axis=1)
df_clean['AM Score'] = df_clean[amotivation_items].sum(axis=1)

# Check the range of the scores
print(df_clean[['IM Score','EM Score','AM Score']].describe())



## Converting Frequency Responses and Computing Eating Habit Scores

The survey records food consumption frequency as categorical strings (e.g. "1 time", "4 times").
We map these strings to numeric counts and then compute composite scores for healthy eating (HE), neutral/mixed diet (NM), and unhealthy eating (UE).


In [ ]:

# Mapping for converting categorical frequency responses to numeric values
frequency_mapping = {
    'Never': 0,
    'almost never': 1,
    '1 time': 1,
    '2 times': 2,
    '3 times': 3,
    '4 times': 4,
    '5 times': 5,
    '5 times or more': 5,
    '6 or 7 times': 5,
    'fairly often': 3,
    'sometimes': 2,
    'very often': 4
}

# Apply the mapping to all eating-related columns
eating_columns = healthy_eating + neutral_items + unhealthy_eating
df_clean[eating_columns] = df_clean[eating_columns].replace(frequency_mapping)

# Compute healthy, neutral/mixed, and unhealthy eating scores
df_clean['HE Score'] = df_clean[healthy_eating].sum(axis=1)
df_clean['NM Score'] = df_clean[neutral_items].sum(axis=1)
df_clean['UE Score'] = df_clean[unhealthy_eating].sum(axis=1)

# Review the distribution of eating scores
print(df_clean[['HE Score','NM Score','UE Score']].describe())



## Computing the Perceived Stress (PS) Score

Perceived stress items are recoded to numeric values:
- "never" = 0, "almost never" = 1, "sometimes" = 2, "fairly often" = 3, "very often" = 4.

We then sum across the ten items to obtain the Perceived Stress score.


In [ ]:

# Mapping for perceived stress responses
stress_mapping = {
    'never': 0,
    'almost never': 1,
    'sometimes': 2,
    'fairly often': 3,
    'very often': 4
}

# Apply the mapping to the perceived stress items
df_clean[perceived_stress_items] = df_clean[perceived_stress_items].replace(stress_mapping)

# Sum perceived stress items to compute the PS Score
df_clean['PS Score'] = df_clean[perceived_stress_items].sum(axis=1)

print(df_clean['PS Score'].describe())



## Finalizing the Dataset

To simplify further analyses and reduce memory usage, we drop the original question columns after computing scores.
We also convert numeric columns to the `int8` type when appropriate.


In [ ]:

# Define all raw question columns to remove after computing scores
columns_to_drop = (
    intrinsic_motivation +
    extrinsic_motivation +
    amotivation_items +
    healthy_eating +
    neutral_items +
    unhealthy_eating +
    k6_items +
    sm_pos +
    sm_neg +
    rs_items +
    fies_cols +
    perceived_stress_items
)

# Drop the specified columns (in-place) from the dataframe
df_clean.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')

# Convert all numeric columns to int8 to save memory
numeric_cols = df_clean.select_dtypes(include='number').columns

# Convert FIES Score separately to avoid downcasting from float
if 'FIES_Score' in numeric_cols:
    df_clean['FIES_Score'] = pd.to_numeric(df_clean['FIES_Score'], errors='coerce').astype('int8')

# Convert other score columns to int8
score_columns = ['SMM_Score','K6_Score','PS Score','HE Score','NM Score','UE Score','IM Score','EM Score','AM Score','RS_Score']
for col in score_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype('int8')

print('Final dataframe information:')
df_clean.info()



## Additional Derived Features

We derive two binary features based on respondents' behavior and location:

1. **`Do you follow a diet?`** – derived from the dietary style question; 'i don’t follow a specific diet' → 'no', otherwise 'yes'.
2. **`What country do you live in ?`** – a simplified geographic indicator distinguishing respondents from the Americas vs. non‑Americas.

We also derive a **Healthy Lifestyle (HL) Score** using responses to physical activity and diet adherence questions.
After computing the score, we drop the original columns.


In [ ]:

# Derive a binary feature indicating whether the respondent follows any diet

df_clean['Do you follow a diet?'] = df_clean['What is your typical dietary style?'].apply(
    lambda x: 'no' if x == "i don’t follow a specific diet" else 'yes'
)

# Simplify the geographic region into America vs Non-America
america_regions = ['North America/Central America', 'South America']
df_clean['What country do you live in ?'] = df_clean['What region do you live in ?'].apply(
    lambda region: 'America' if region in america_regions else 'Non-America'
)

# Define physical activity columns for Healthy Lifestyle Score
healthy_lifestyle_items = [
    'How many times per week, on average, do you engage in physical activity for at least 20 minutes a day?',
    'During the past 7 days, how many days did you exercise for at least 20 minutes?',
    'Do you follow a diet?'
]

# Mapping of physical activity responses to numeric values
activity_mapping = {
    'None of the time': 0,
    '1 time': 1,
    '2 times': 2,
    '3 times': 3,
    '4 times': 4,
    '5 times': 5,
    '6-7 times': 6,
    'yes': 1,
    'no': 0
}

# Convert activity and diet responses to numeric
for col in healthy_lifestyle_items:
    df_clean[col] = df_clean[col].replace(activity_mapping).astype('int8')

# Compute the Healthy Lifestyle (HL) Score
df_clean['HL Score'] = df_clean[healthy_lifestyle_items].sum(axis=1)

# Drop the individual activity columns
df_clean.drop(columns=healthy_lifestyle_items, axis=1, inplace=True, errors='ignore')

print(df_clean[['HL Score']].describe())



## Saving the Cleaned Dataset

After all transformations and feature engineering, we save the final cleaned DataFrame to a CSV file for subsequent analysis.


In [ ]:

# Specify the filename for the cleaned dataset
output_csv = 'cleaned_nutrition_survey.csv'

# Save the cleaned DataFrame to disk (without the index)
df_clean.to_csv(output_csv, index=False)

print(f'Cleaned dataset saved to {output_csv}')



# Research Questions

Now that we have a clean dataset with computed scores, we can explore various research questions.
Some examples include:

1. **Does food insecurity (FIES_Score) correlate with psychological distress (K6_Score)?**
   We can calculate correlation coefficients or compare average K6 scores across different levels of food insecurity.

2. **Does resilience (RS_Score) differ across geographic regions?**
   By grouping respondents by the simplified `What country do you live in ?` column, we can test whether resilience scores vary significantly between America and Non-America.

Feel free to extend these questions or formulate your own hypotheses based on the computed features.
